# 📈 Baseline Strategy Analysis

**Project:** Deep Reinforcement Learning for Dynamic Asset Allocation  
**Notebook:** 02 - Baseline Strategies  
**Purpose:** Analyze and compare classical portfolio allocation strategies

---

## 1. Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from baselines import (
    MertonStrategy,
    MeanVarianceStrategy,
    EqualWeightStrategy,
    BuyAndHoldStrategy,
    RiskParityStrategy
)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Imports complete")

## 2. Load Backtest Results

In [ ]:
# Load comparison results
results_path = '../simulations/baseline_results/baseline_comparison.csv'
comparison_df = pd.read_csv(results_path)

print("\n" + "="*100)
print("BASELINE STRATEGY PERFORMANCE (2014-2024)")
print("="*100)
print(comparison_df.to_string(index=False))
print("="*100)

## 3. Load Individual Strategy Results

In [ ]:
# Load portfolio values for each strategy
strategies = ['merton', 'mean_variance', 'equal_weight', 'buy_and_hold', 'risk_parity']
portfolio_values = {}
weights_history = {}

for strategy in strategies:
    pv_path = f'../simulations/baseline_results/{strategy}/portfolio_values.csv'
    wh_path = f'../simulations/baseline_results/{strategy}/weights_history.csv'
    
    portfolio_values[strategy] = pd.read_csv(pv_path)
    weights_history[strategy] = pd.read_csv(wh_path)

print(f"\n✅ Loaded {len(strategies)} strategy results")

## 4. Equity Curves Comparison

In [ ]:
# Plot all equity curves
fig, ax = plt.subplots(figsize=(14, 7))

strategy_labels = {
    'merton': 'Merton',
    'mean_variance': 'Mean-Variance',
    'equal_weight': 'Equal-Weight',
    'buy_and_hold': 'Buy-and-Hold',
    'risk_parity': 'Risk Parity'
}

for strategy in strategies:
    pv = portfolio_values[strategy]['portfolio_value'].values
    ax.plot(pv, label=strategy_labels[strategy], linewidth=2)

ax.set_title('Baseline Strategy Equity Curves', fontsize=16, fontweight='bold')
ax.set_xlabel('Trading Days', fontsize=12)
ax.set_ylabel('Portfolio Value ($)', fontsize=12)
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_yscale('log')  # Log scale to see all strategies

plt.tight_layout()
plt.show()

## 5. Risk-Return Analysis

In [ ]:
# Risk-return scatter
fig, ax = plt.subplots(figsize=(10, 7))

annual_returns = comparison_df['Annual Return (%)'].values
volatilities = comparison_df['Volatility (%)'].values
names = comparison_df['Strategy'].values
sharpes = comparison_df['Sharpe Ratio'].values

scatter = ax.scatter(volatilities, annual_returns, 
                    s=400, c=sharpes, cmap='RdYlGn',
                    alpha=0.6, edgecolors='black', linewidth=2)

# Add labels
for i, name in enumerate(names):
    ax.annotate(name, (volatilities[i], annual_returns[i]),
               xytext=(7, 7), textcoords='offset points', 
               fontsize=10, fontweight='bold')

# Colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Sharpe Ratio', fontsize=11)

ax.set_title('Risk-Return Profile (Baseline Strategies)', fontsize=16, fontweight='bold')
ax.set_xlabel('Volatility (% p.a.)', fontsize=12)
ax.set_ylabel('Annual Return (%)', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Weight Allocation Analysis

In [ ]:
# Plot allocation patterns for dynamic strategies
dynamic_strategies = ['merton', 'mean_variance', 'risk_parity']

fig, axes = plt.subplots(len(dynamic_strategies), 1, figsize=(14, 12), sharex=True)

asset_names = ['SPY', 'TLT', 'GLD', 'BTC']

for i, strategy in enumerate(dynamic_strategies):
    ax = axes[i]
    weights = weights_history[strategy]
    
    # Stacked area plot
    ax.stackplot(range(len(weights)), 
                weights.iloc[:, 0], weights.iloc[:, 1], 
                weights.iloc[:, 2], weights.iloc[:, 3],
                labels=asset_names, alpha=0.8)
    
    ax.set_title(f'{strategy_labels[strategy]} - Asset Allocation', 
                fontsize=12, fontweight='bold')
    ax.set_ylabel('Weight', fontsize=10)
    ax.legend(loc='upper left', fontsize=9, ncol=4)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, 1)

axes[-1].set_xlabel('Trading Days', fontsize=12)

plt.tight_layout()
plt.show()

## 7. Turnover Analysis

In [ ]:
# Compare turnover rates
fig, ax = plt.subplots(figsize=(10, 6))

turnovers = comparison_df['Avg Turnover (%)'].values
strategy_names = comparison_df['Strategy'].values

colors = ['red' if t > 20 else 'green' for t in turnovers]

bars = ax.barh(strategy_names, turnovers, color=colors, alpha=0.7, edgecolor='black')

# Add value labels
for i, (name, turnover) in enumerate(zip(strategy_names, turnovers)):
    ax.text(turnover + 1, i, f'{turnover:.2f}%', 
           va='center', fontsize=10, fontweight='bold')

ax.set_title('Average Daily Turnover by Strategy', fontsize=14, fontweight='bold')
ax.set_xlabel('Average Turnover (%)', fontsize=12)
ax.grid(True, alpha=0.3, axis='x')

# Add annotation
ax.text(0.98, 0.98, 'Red: High turnover\nGreen: Low turnover',
       transform=ax.transAxes, fontsize=9,
       verticalalignment='top', horizontalalignment='right',
       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("\n💡 Turnover Insights:")
print(f"   - Buy-and-Hold has 0% turnover (no rebalancing)")
print(f"   - Equal-Weight has low turnover (~3%, only rebalances to maintain equal weights)")
print(f"   - Mean-Variance has highest turnover (~52%, actively responds to market changes)")
print(f"   - High turnover = higher transaction costs but potentially better risk management")

## 8. Drawdown Analysis

In [ ]:
# Calculate and plot drawdowns
fig, ax = plt.subplots(figsize=(14, 7))

for strategy in strategies:
    pv = portfolio_values[strategy]['portfolio_value'].values
    
    # Calculate drawdown
    running_max = np.maximum.accumulate(pv)
    drawdown = (pv - running_max) / running_max * 100
    
    ax.plot(drawdown, label=strategy_labels[strategy], linewidth=1.5, alpha=0.8)

ax.set_title('Portfolio Drawdowns Over Time', fontsize=16, fontweight='bold')
ax.set_xlabel('Trading Days', fontsize=12)
ax.set_ylabel('Drawdown (%)', fontsize=12)
ax.legend(loc='lower left', fontsize=11)
ax.grid(True, alpha=0.3)
ax.axhline(0, color='black', linestyle='--', linewidth=1)

# Highlight major drawdown period (COVID-19)
ax.axvspan(1300, 1400, alpha=0.2, color='red', label='COVID-19 Crash Period (approx.)')

plt.tight_layout()
plt.show()

# Max drawdown comparison
print("\n" + "="*60)
print("MAXIMUM DRAWDOWN COMPARISON")
print("="*60)
for i, row in comparison_df.iterrows():
    print(f"{row['Strategy']:15s}: {row['Max Drawdown (%)']:6.2f}%")
print("="*60)

## 9. Performance Metrics Radar Chart

In [ ]:
# Create radar chart for performance comparison
from math import pi

# Normalize metrics to [0, 1]
metrics = ['Sharpe Ratio', 'Sortino Ratio', 'Calmar Ratio', 'Win Rate (%)']
normalized_data = {}

for metric in metrics:
    values = comparison_df[metric].values
    normalized = (values - values.min()) / (values.max() - values.min())
    normalized_data[metric] = normalized

# Add inverted max drawdown (lower is better)
dd_values = comparison_df['Max Drawdown (%)'].values
normalized_data['Risk Control'] = 1 - (dd_values - dd_values.min()) / (dd_values.max() - dd_values.min())
metrics.append('Risk Control')

# Setup radar
num_vars = len(metrics)
angles = [n / float(num_vars) * 2 * pi for n in range(num_vars)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

for i, strategy in enumerate(strategies):
    values = [normalized_data[metric][i] for metric in metrics]
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, label=strategy_labels[strategy])
    ax.fill(angles, values, alpha=0.15)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(metrics, fontsize=11)
ax.set_ylim(0, 1)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=9)
ax.set_title('Baseline Strategies - Performance Metrics (Normalized)', 
            fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=10)
ax.grid(True)

plt.tight_layout()
plt.show()

## 10. Strategy Rankings

In [ ]:
# Rank strategies by different metrics
ranking_metrics = [
    ('Total Return (%)', False),  # False = higher is better
    ('Sharpe Ratio', False),
    ('Sortino Ratio', False),
    ('Max Drawdown (%)', True),   # True = lower is better
    ('Avg Turnover (%)', True)
]

print("\n" + "="*80)
print("STRATEGY RANKINGS BY METRIC")
print("="*80)

for metric, ascending in ranking_metrics:
    ranked = comparison_df.sort_values(metric, ascending=ascending)[['Strategy', metric]]
    print(f"\n{metric}:")
    for i, row in ranked.iterrows():
        print(f"  {ranked.index.tolist().index(i)+1}. {row['Strategy']:15s}: {row[metric]:8.2f}")

print("\n" + "="*80)

## 11. Key Findings

### Performance Summary:

1. **Best Risk-Adjusted Returns:** Equal-Weight (Sharpe 0.845)
   - Simple 1/N allocation performs remarkably well
   - Low turnover (3.27%) keeps transaction costs minimal
   - Consistently good across multiple metrics

2. **Highest Total Returns:** Mean-Variance (1442.61%)
   - Active optimization captures market opportunities
   - Higher turnover (52.30%) implies higher transaction costs
   - More volatile but highest absolute gains

3. **Best Downside Protection:** Risk Parity (29.44% max drawdown)
   - Volatility-weighting provides good risk control
   - Moderate turnover (8.76%)
   - Excellent Sharpe (0.723) and Sortino (1.074) ratios

4. **Classical Benchmark:** Merton (Sharpe 0.778)
   - Theoretical optimal control performs well
   - Assumes constant parameters (limitation)
   - Good baseline for comparison

5. **Simplest Strategy:** Buy-and-Hold (Sharpe 0.597)
   - Zero turnover = zero transaction costs
   - Passive but reasonable performance
   - Underperforms active strategies

### Implications for RL:

- **Target to beat:** Equal-Weight's Sharpe ratio of 0.845
- **Turnover consideration:** RL agents should balance alpha generation vs. transaction costs
- **Regime adaptation:** Classical strategies with fixed parameters leave room for RL improvement
- **Risk management:** RL should aim for Risk Parity's drawdown control while improving returns

---

**Next:** Train RL agents and compare against these baselines (Notebooks 03-04)
